In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Visualization

### Let's check number of checks for every mental disorder

In [ ]:
disorder_prevalence_df = pd.read_csv('/kaggle/input/mental-health/7- number-of-countries-with-primary-data-on-prevalence-of-mental-illnesses-in-the-global-burden-of-disease-study.csv')

In [ ]:
disorder_prevalence_df.describe()

In [ ]:
disorder_prevalence_df.head(10)

In [ ]:
disorder_prevalence_df.rename(columns = {'Number of countries with primary data on prevalence of mental disorders':'Total'}, inplace = True)

In [ ]:
disorder_prevalence_df.sort_values(by=['Total'], ascending=False)[:5]

So the top 5 are:
* ADHD
* Cannabis
* MDD
* Substance Abuse (Cocaine)
* Alcohol

### Let's see for which of these there are information about Iran.

Later we can look at the relationship of these data.

In [ ]:
mental_df = pd.read_csv('/kaggle/input/mental-health/1- mental-illnesses-prevalence.csv')

In [ ]:
mental_df.head(10)

In [ ]:
mental_df.describe()

In [ ]:
# Renaming the disorder for more pandas firendly names
# Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized   -> Schizophrenia
# Depressive disorders (share of population) - Sex: Both - Age: Age-standardized      -> Depressive
# Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized         -> Anxiety
# Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized         -> BPD
# Eating disorders (share of population) - Sex: Both - Age: Age-standardized          -> Eating


mental_df.rename(columns = {'Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized':'Schizophrenia'}, inplace = True)
mental_df.rename(columns = {'Depressive disorders (share of population) - Sex: Both - Age: Age-standardized':'Depressive'}, inplace = True)
mental_df.rename(columns = {'Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized':'Anxiety'}, inplace = True)
mental_df.rename(columns = {'Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized':'BPD'}, inplace = True)
mental_df.rename(columns = {'Eating disorders (share of population) - Sex: Both - Age: Age-standardized':'Eating'}, inplace = True)

In [ ]:
countries = mental_df.Entity.values

In [ ]:
iran_idx = np.where(countries == 'Iran')[0]

In [ ]:
iran_block = mental_df.loc[iran_idx].copy()

In [ ]:
iran_block.rename(columns = {'Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized':'Schizophrenia'}, inplace = True)
iran_block.rename(columns = {'Depressive disorders (share of population) - Sex: Both - Age: Age-standardized':'Depressive'}, inplace = True)
iran_block.rename(columns = {'Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized':'Anxiety'}, inplace = True)
iran_block.rename(columns = {'Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized':'BPD'}, inplace = True)
iran_block.rename(columns = {'Eating disorders (share of population) - Sex: Both - Age: Age-standardized':'Eating'}, inplace = True)

In [ ]:
print(iran_block['Year'].min())
print(iran_block['Year'].max())

In [ ]:
iran_block.columns

In [ ]:
iran_block

In [ ]:
plt.scatter(np.arange(len(iran_block)), iran_block['Depressive'])

In [ ]:
data_to_plot = iran_block.drop(['Entity', 'Code', 'Year'], axis=1)

# Determine the number of columns to plot, 36
num_cols = len(data_to_plot.columns)

# Calculate the number of rows and columns for the subplots
num_rows = (num_cols + 2) // 3
num_cols_per_subplot = min(num_cols, 5)
print(f'Rows: {num_rows} and Columns: {num_cols_per_subplot}')

In [ ]:
# Create subplots
fig, axes = plt.subplots(num_rows, num_cols_per_subplot, figsize=(18, 2 * num_rows))

# Flatten the axes for iteration
axes = axes.flatten()

# Create box plots for each column
for i, col in enumerate(data_to_plot.columns):
    # The main magic happens here:
    sns.boxplot(x=data_to_plot[col], ax=axes[i])
    axes[i].set_title(col)
    axes[i].set_xticks([])  # Remove x-axis labels for better readability
    
# Remove any empty subplots
for i in range(num_cols, num_rows * num_cols_per_subplot):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# Schizophrenia	Depressive	Anxiety	BPD	Eating
data_to_plot = iran_block.drop(['Entity', 'Code', 'Year'], axis=1)

# Determine the number of columns to plot, 36
num_cols = len(data_to_plot.columns)

# Calculate the number of rows and columns for the subplots
num_rows = (num_cols + 2) // 3
num_cols_per_subplot = min(num_cols, 3)
print(f'Rows: {num_rows} and Columns: {num_cols_per_subplot}')


fig, axes = plt.subplots(num_rows, num_cols_per_subplot, figsize=(19, 4 * num_rows))

# Flatten the axes for iteration
axes = axes.flatten()

for i, col in enumerate(data_to_plot.columns):
    # The main magic happens here:
    # sns.lineplot(data=df, x='Year',y='Schizophrenia')
    sns.lineplot(data = iran_block, x='Year', y=col, ax=axes[i])
    axes[i].set_title(col)
#     axes[i].set_xticks([])  # Remove x-axis labels for better readability

for i in range(num_cols, num_rows * num_cols_per_subplot):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

The problem is that we don't know the how to assume the population for each year from 1990 until 2019. It would be ignorant for me to assume 80M and if i can't get the population for each year than I have to assume linear growth.

In [ ]:
iran_pop = pd.read_csv('/kaggle/input/iran-population-from-1950-to-2023/iran_population.csv')
iran_pop

In [ ]:
iran_block.Year.min()

In [ ]:
iran_pop.date = pd.to_datetime(iran_pop.date)

In [ ]:
iran_pop.date

In [ ]:
iran_pop_select = iran_pop[(iran_pop['date'] > '1990') & (iran_pop['date'] < '2020')]

In [ ]:
 iran_pop_select.date = iran_pop_select.date.dt.year

In [ ]:
iran_pop_select = iran_pop_select.reset_index(drop=True)

In [ ]:
iran_block = iran_block.reset_index(drop=True)

In [ ]:
iran_block

In [ ]:
iran_block['Population'] = iran_pop_select.iloc[:, 1]
iran_block.drop(['Entity', 'Code'], axis=1, inplace=True)
iran_block

In [ ]:
iran_block.to_csv('iran-mental-health_data.csv')